In [4]:
import os

PATH = '/home/stadlerm/Development/Projects/DreamChallenge/RA2_alpine_lads'

os.chdir(PATH)

In [5]:
from utils.config import Config

config = Config()

In [6]:
import os
import pandas as pd

training_csv = os.path.join(config.train_location, 'training.csv')

training_data = pd.read_csv(training_csv)

In [9]:
import re

import numpy as np

# Prepare outcome vector
column_types = [
    ['hands_narrowing', '.*H_.*_J.*'], 
    ['hands_erosion', '.*H_.*_E.*'], 
    ['feet_narrowing', '.*F_.*_J.*'],
    ['feet_erosion', '.*F_.*_E.*']
]

for column_type in column_types:
    file_post_fix = column_type[0]
    column_reg = column_type[1]
    
    p = re.compile(column_reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    values = training_data[columns]
    vals, counts = np.unique(values, return_counts = True)
    
    y = np.zeros((training_data.shape[0], vals.size))
    
    for idx, val in enumerate(vals):
        r_idx = np.where(values == val)[0]
        
        y[r_idx, idx] = 1
        
    y_t = y[:, 1]

    for idx in range(1, vals.size):
        y_t = np.logical_and(y_t, y[:, idx])
    
    print(np.count_nonzero(y_t))

6
2
1
0


In [10]:
import functools

y_t = y[:, 0]

for idx in range(1, 5):
    y_t = np.logical_and(y_t, y[:, idx])
    
print(np.count_nonzero(y_t))

4


In [13]:
# Regex for hand narrowing, erosion, and feet narrowing columns
column_reg = ['.*F_.*_J.*']

for idx, reg in enumerate(column_reg):
    p = re.compile(reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    values = training_data[columns].to_numpy()
    vals, c = np.unique(values, return_counts = True)
    
    sorted_idx = np.argsort(c)[::-1]
    
    y = np.zeros(training_data.shape[0])
    for idx in sorted_idx[1:]:
        y[np.where(values == vals[idx])[0]] = vals[idx]

In [15]:
print(np.unique(y, return_counts = True))

(array([0., 1., 2., 3., 4.]), array([257,   7,  47,  28,  29]))


In [18]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(np.arange(training_data.shape[0]), test_size = 0.25, stratify = y)

In [19]:
print(np.unique(y[train_idx], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([193,   5,  35,  21,  22]))


In [20]:
print(np.unique(y[test_idx], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([64,  2, 12,  7,  7]))


In [21]:
train_patient_id = training_data.iloc[train_idx]['Patient_ID'].to_numpy()
test_patient_id = training_data.iloc[test_idx]['Patient_ID'].to_numpy()

In [22]:
print(train_patient_id)

['UAB003' 'UAB060' 'UAB580' 'UAB042' 'UAB130' 'UAB407' 'UAB417' 'UAB233'
 'UAB122' 'UAB002' 'UAB110' 'UAB701' 'UAB013' 'UAB165' 'UAB504' 'UAB297'
 'UAB672' 'UAB422' 'UAB537' 'UAB036' 'UAB524' 'UAB403' 'UAB049' 'UAB192'
 'UAB028' 'UAB100' 'UAB383' 'UAB342' 'UAB679' 'UAB124' 'UAB123' 'UAB719'
 'UAB131' 'UAB067' 'UAB711' 'UAB143' 'UAB171' 'UAB160' 'UAB716' 'UAB396'
 'UAB159' 'UAB366' 'UAB363' 'UAB045' 'UAB041' 'UAB271' 'UAB399' 'UAB487'
 'UAB244' 'UAB216' 'UAB700' 'UAB491' 'UAB536' 'UAB499' 'UAB551' 'UAB486'
 'UAB630' 'UAB111' 'UAB032' 'UAB703' 'UAB675' 'UAB125' 'UAB085' 'UAB007'
 'UAB697' 'UAB453' 'UAB481' 'UAB228' 'UAB200' 'UAB659' 'UAB179' 'UAB105'
 'UAB052' 'UAB326' 'UAB549' 'UAB394' 'UAB490' 'UAB031' 'UAB374' 'UAB033'
 'UAB379' 'UAB080' 'UAB062' 'UAB144' 'UAB070' 'UAB198' 'UAB441' 'UAB107'
 'UAB625' 'UAB538' 'UAB333' 'UAB182' 'UAB617' 'UAB449' 'UAB720' 'UAB571'
 'UAB357' 'UAB329' 'UAB494' 'UAB577' 'UAB239' 'UAB199' 'UAB649' 'UAB184'
 'UAB135' 'UAB574' 'UAB012' 'UAB098' 'UAB464' 'UAB5

In [23]:
print(test_patient_id)

['UAB196' 'UAB061' 'UAB552' 'UAB071' 'UAB234' 'UAB717' 'UAB601' 'UAB008'
 'UAB056' 'UAB519' 'UAB347' 'UAB030' 'UAB704' 'UAB150' 'UAB576' 'UAB557'
 'UAB389' 'UAB185' 'UAB488' 'UAB138' 'UAB420' 'UAB088' 'UAB024' 'UAB127'
 'UAB323' 'UAB436' 'UAB163' 'UAB427' 'UAB103' 'UAB671' 'UAB225' 'UAB607'
 'UAB573' 'UAB243' 'UAB517' 'UAB337' 'UAB197' 'UAB180' 'UAB640' 'UAB428'
 'UAB115' 'UAB706' 'UAB666' 'UAB277' 'UAB288' 'UAB461' 'UAB500' 'UAB057'
 'UAB251' 'UAB005' 'UAB639' 'UAB683' 'UAB208' 'UAB653' 'UAB620' 'UAB023'
 'UAB128' 'UAB550' 'UAB367' 'UAB688' 'UAB702' 'UAB345' 'UAB384' 'UAB626'
 'UAB411' 'UAB482' 'UAB040' 'UAB101' 'UAB501' 'UAB210' 'UAB352' 'UAB348'
 'UAB562' 'UAB609' 'UAB530' 'UAB075' 'UAB698' 'UAB545' 'UAB099' 'UAB654'
 'UAB535' 'UAB086' 'UAB331' 'UAB021' 'UAB035' 'UAB539' 'UAB393' 'UAB162'
 'UAB069' 'UAB413' 'UAB388' 'UAB065']


In [32]:
print(np.intersect1d(train_patient_id, test_patient_id))

[]


In [24]:
hands_dataframe = pd.read_csv('./data/hand_joint_data.csv')
feet_dataframe = pd.read_csv('./data/feet_joint_data.csv')

In [25]:
hand_idx = [image_name.split('-')[0] in test_patient_id for image_name in hands_dataframe['image_name']]
feet_idx = [image_name.split('-')[0] in test_patient_id for image_name in feet_dataframe['image_name']]

In [26]:
hands_dataframe.iloc[np.where(np.logical_not(hand_idx))[0]].to_csv('./data/hand_joint_data_train.csv')
feet_dataframe.iloc[np.where(np.logical_not(feet_idx))[0]].to_csv('./data/feet_joint_data_train.csv')

hands_dataframe.iloc[np.where(hand_idx)[0]].to_csv('./data/hand_joint_data_test.csv')
feet_dataframe.iloc[np.where(feet_idx)[0]].to_csv('./data/feet_joint_data_test.csv')

In [27]:
feet_data = pd.read_csv('./data/feet_joint_data_train.csv')
feet_data_val = pd.read_csv('./data/feet_joint_data_test.csv')

In [28]:
np.unique(feet_data['narrowing_0'], return_counts = True)

(array([0., 1., 2., 3., 4.]), array([3021,    5,  142,   72,   72]))

In [29]:
np.unique(feet_data_val['narrowing_0'], return_counts = True)

(array([0., 1., 2., 3., 4.]), array([985,   2,  55,  34,  28]))

In [30]:
x = np.count_nonzero(feet_data_val['narrowing_0'] == 0)

In [31]:
print(x / feet_data_val.shape[0])

0.8922101449275363
